# 🧠 Self-Reflecting Agentic AI (No APIs, No LLMs)

This tutorial demonstrates a **real, runnable self-reflecting agent** using:

- Pure Python
- Local ML model (scikit-learn)
- Agentic loop inspired by **LangGraph**

💡 The agent **observes its own past actions**, reflects on failures, and **changes behavior automatically**.

## 🔍 What is Self-Reflection in Agentic AI?

A self-reflecting agent:

1. Takes actions
2. Stores outcomes in memory
3. Evaluates past performance
4. Modifies future decisions

This is **core to agentic systems**, planning loops, and autonomous ML pipelines.

In [ ]:
# Imports
from dataclasses import dataclass, field
from typing import List, Dict, Any

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 🧩 Agent State (LangGraph-Style)

The **state** flows between nodes.

- `history` → memory of past actions
- `strategy` → current decision policy
- `done` → termination signal

In [ ]:
@dataclass
class AgentState:
    history: List[Dict[str, Any]] = field(default_factory=list)
    strategy: str = "default"
    done: bool = False

## ⚙️ Action Node: Train a Local ML Model

This node represents an **agent action**.

The agent trains a model differently based on its **current strategy**.

In [ ]:
def train_model(state: AgentState):
    X, y = make_classification(
        n_samples=200,
        n_features=5,
        class_sep=0.5 if state.strategy == "default" else 1.5,
        random_state=42
    )

    model = LogisticRegression()
    model.fit(X[:150], y[:150])

    preds = model.predict(X[150:])
    acc = accuracy_score(y[150:], preds)

    state.history.append({
        "action": "train_model",
        "strategy": state.strategy,
        "accuracy": acc
    })

    print(f"[Action] Strategy={state.strategy} | Accuracy={acc:.2f}")
    return state

## 🔁 Reflection Node (The Brain)

This node **reads the agent’s own history** and decides:

- Should I change strategy?
- Should I stop?

This is **true self-reflection**, not hardcoded control flow.

In [ ]:
def reflect(state: AgentState):
    last = state.history[-1]
    acc = last["accuracy"]

    print(f"[Reflection] Evaluating accuracy: {acc:.2f}")

    if acc < 0.75:
        print("[Reflection] Performance low → switching strategy")
        state.strategy = "improved"
    else:
        print("[Reflection] Performance acceptable → stopping")
        state.done = True

    state.history.append({
        "action": "reflect",
        "decision": state.strategy,
        "stop": state.done
    })

    return state

## 🧭 Control Node (Graph Edge Logic)

This replaces LangGraph conditional edges.

In [ ]:
def should_continue(state: AgentState):
    return not state.done

## 🔄 Agentic Execution Loop

This simulates a **LangGraph execution**:

`Action → Reflection → Decision → Loop`

In [ ]:
def run_agent():
    state = AgentState()
    step = 0

    while should_continue(state) and step < 3:
        print(f"\n=== Step {step + 1} ===")
        state = train_model(state)
        state = reflect(state)
        step += 1

    return state

## ▶️ Run the Self-Reflecting Agent

In [ ]:
final_state = run_agent()

print("\n--- FINAL AGENT MEMORY ---")
for item in final_state.history:
    print(item)

## ✅ Why This Is a Real Agentic System

- ✔ Reads its own memory
- ✔ Reflects on failures
- ✔ Changes behavior
- ✔ Stops autonomously
- ✔ No LLMs, no APIs, no magic

🎯 This pattern scales directly to **LangGraph + LLM reflection** later.